Statistics and overview

Data cleaning

In [36]:
import pandas as pd
import re
import numpy as np
import datetime

months_map = {
    "stycznia": "January",
    "lutego": "February",
    "marca": "March",
    "kwietnia": "April",
    "maja": "May",
    "czerwca": "June",
    "lipca": "July",
    "sierpnia": "August",
    "września": "September",
    "października": "October",
    "listopada": "November",
    "grudnia": "December"
}

def polish_to_eng(date_string):
    for pl_month, en_month in months_map.items():
        if pl_month in date_string:
            return date_string.replace(pl_month,en_month)
    return date_string


with open("./../data/data_scrapped/interim/flag_date_scrapping.txt", "r") as f:
    lines = f.readlines()
    dates = [i for i,item in enumerate(lines) if "last scrapping date" in item.lower()]
    new_data_line=dates[1]+1
    print(lines[new_data_line])  # Print the date for verification
    new_data=lines[new_data_line]
    new_data=new_data[:-1]
 
    old_data_line=dates[0]+1
    old_data=lines[old_data_line]
    old_data=old_data[:-1]
    print(new_data) ############## Print the date for verification
    print(old_data)  # Print the date for verification

scrapping_date = new_data




dataframe = pd.read_csv("./../data/data_scrapped/reads/"+new_data+".csv")

dataframe = dataframe.drop_duplicates()
dataframe = dataframe.drop(
    columns=[
        "ID",
        "Title",
    ]
)


dataframe["Refreshed"]=dataframe["Location"].astype(str).apply(lambda x: True if "Od" in x else False)

dataframe[["Location", "Date"]] = dataframe["Location"].str.extract(r"^(.*?)(\d.*)$")
dataframe["Location"] = dataframe["Location"].str.split("-").str[0].str.strip()

dataframe["Negotiations"] = dataframe["Price"].str.contains("do", na=False)
dataframe["Price"] = dataframe["Price"].astype(str).str.replace(",",".",regex=False)
dataframe["Price"] = dataframe["Price"].str.split(" zł").str[0].str.strip()

dataframe["Price"] = dataframe["Price"].str.split(" złdo").str[0].str.strip()
dataframe["Price"] = dataframe["Price"].astype(str).str.replace(" zł", "", regex=False)
# dataframe["Negotiations"] = dataframe["Price"].str.contains(
#     r"[a-zA-Z]", regex=True, na=False
# )
# dataframe["Price"]=dataframe["Price"].str.replace(r'[^0-9]',"",regex=True)
# dataframe["Price"]=pd.to_numeric(dataframe["Price"],errors="coerce")

dataframe["Area"] = dataframe["Area"].str.split("m²").str[0].str.strip()
dataframe["Area"] = dataframe["Area"].astype(str).str.replace("m²", "", regex=False)
dataframe["Area"] = dataframe["Area"].astype(str).str.replace(",",".",regex=False)
dataframe["Area"]=pd.to_numeric(dataframe["Area"],errors="coerce")

mask3 = (
    dataframe["Type_of_Agreement"].isin(["sprzedaz"])
    & dataframe["Pets"].isnull()
)
dataframe.loc[mask3, "Pets"] = "yes"
dataframe["Pets"] = dataframe["Pets"].fillna("no")

mask4 = (
    dataframe["Type_of_Agreement"].isin(["sprzedaz"])
    & (dataframe["Parking_Space"].isnull()) & (dataframe["Dwelling_Type"].isin(["detached_house", "terraced_house"]))  
)

dataframe.loc[mask4, "Parking_Space"] = "yes"

dataframe["Parking_Space"] = dataframe["Parking_Space"].fillna("no")
dataframe["Parking_Space"] = (
    dataframe["Parking_Space"].astype(str).str.replace("lack", "no", regex=False)
)

dataframe["Furnishing"] = dataframe["Furnishing"].fillna("no")

mask = (
    dataframe["Dwelling_Type"].isin(["terraced_house", "detached_house"])
    & dataframe["Level"].isnull()
)
dataframe.loc[mask, "Level"] = "0"

mask_2 = (
    dataframe["Dwelling_Type"].isin(["terraced_house", "detached_house", "tenement"])
    & dataframe["Elevator"].isnull()
)
dataframe.loc[mask_2, "Elevator"] = "no"



dataframe = dataframe.dropna(subset=["Seller", "Dwelling_Type", "Rooms_Number"])

dataframe["Pets"]=dataframe["Pets"].str.lower().map({"yes":True,"no":False})
dataframe["Furnishing"]=dataframe["Furnishing"].str.lower().map({"yes":True,"no":False})
dataframe["Elevator"]=dataframe["Elevator"].str.lower().map({"yes":True,"no":False})



# Levels to numerics
dataframe["Level"] = dataframe["Level"].astype(str).str.replace("10+", "11")
dataframe["Level"] = dataframe["Level"].astype(str).str.replace("nan", "")
dataframe["Level"]=pd.to_numeric(dataframe["Level"],errors="coerce")
dataframe["Level"]=dataframe["Level"].fillna(-1).astype(int)
dataframe.loc[(dataframe["Level"] == -1) & (dataframe["Elevator"]==1),"Level"]=1
dataframe.loc[(dataframe["Level"]==-1)&(dataframe["Dwelling_Type"]=="tenement"),"Level"]=4
dataframe.loc[(dataframe["Level"]==-1),"Level"] = 4
print(dataframe["Level"].unique())


dataframe["Rooms_Number"]=dataframe["Rooms_Number"].str.replace("4+","4",regex=False)
dataframe["Rooms_Number"]=pd.to_numeric(dataframe["Rooms_Number"],errors="coerce")
print(dataframe["Rooms_Number"].unique())

#To do date

# Hour to day of the scrapping
dataframe["Date"]=dataframe["Date"].astype(str).apply(lambda x: scrapping_date if ":" in x else x)
dataframe["Date"]=dataframe["Date"].apply(polish_to_eng)
dataframe["Date"]=pd.to_datetime(dataframe["Date"], dayfirst=True,errors='coerce').dt.date


# all builings in Poland, which have more than 4 levels need to have elevator

# W końcu to jebane elevator

dataframe["Elevator"]=dataframe["Elevator"].fillna(-1).astype(int)
dataframe.loc[(dataframe["Level"]>4) &(dataframe["Elevator"]==-1),"Elevator"] =1
dataframe["Elevator"]=dataframe["Elevator"].replace(-1,0)
# print(dataframe.isnull().sum())
# print(dataframe.info())



#To do : split data by type of agreement, for sale pets all yes, for rent null all no; parking space for sale detached hourse yes, for rent no

2025_07_17_01

2025_07_17_01
2025_07_14_15
[ 2  1  4  0 10  5  3  9  8  6  7 11]
[2 1 3 4]


Number of records count

In [37]:
count = dataframe.shape[0]

with open("./../data/data_scrapped/interim/flag_date_scrapping.txt", "a") as f:
    f.write(f"Downloaded records count: {count}\n")

Older record filtering

In [38]:
from datetime import datetime

with open("./../data/data_scrapped/interim/flag_date_scrapping.txt", "r") as f:
    lines = f.readlines()
    print(lines[2])
last_scrapping_date = lines[2].strip()#or old_data
last_scrapping_date = last_scrapping_date[:-3].replace("_", "-")

print(last_scrapping_date)
# Convert last_scrapping_date to datetime.date for comparison
last_scrapping_date_dt = datetime.strptime(last_scrapping_date, "%Y-%m-%d").date()
dataframe = dataframe[dataframe["Date"] >= last_scrapping_date_dt]
cities = dataframe["City"].unique()

with open("./../data/data_scrapped/interim/flag_date_scrapping.txt", "a") as f:
    f.write(f"Records count after filtering: {dataframe.shape[0]}\n")
    for city in cities:
        count_city = dataframe[dataframe["City"] == city].shape[0]
        f.write(f"City: {city}, Records count: {count_city}\n")

2025_07_14_15

2025-07-14


Send e-mail

In [ ]:
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from email.mime.text import MIMEText
import base64
from datetime import datetime
now = datetime.now()
n = now.strftime("%Y-%m-%d %H:%M:%S")

with open("./../data/data_scrapped/interim/flag_date_scrapping.txt", "r") as f:
    lines = f.readlines()
    subject = lines[int(new_data_line)].strip(), lines[int(new_data_line+2)].strip()
    subject_data = " ".join(subject)  # Join the subject lines into a single string
    # Collect city records for the body
    body_lines = [lines[i].strip() for i in range(int(new_data_line+1), int(len(lines)))]
    body_data = "\n\n".join(body_lines)
    body_data = f"{body_data}\n\n{n}"  # Add timestamp to the body
    print(subject_data)  # Print the subject for verification
    print(body_data)

SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def get_gmail_credentials():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('cred.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

def create_message(to_email, from_email, subject, body):
    message = MIMEText(body)
    message['to'] = to_email
    message['from'] = from_email
    message['subject'] = subject
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw_message}

def send_email_via_gmail_api(subject, body, to_email, from_email):
    creds = get_gmail_credentials()
    service = build('gmail', 'v1', credentials=creds)

    message = create_message(to_email, from_email, subject, body)
    sent_message = service.users().messages().send(userId='me', body=message).execute()
    print(f"Message sent! ID: {sent_message['id']}")




# Example usage
send_email_via_gmail_api(
    subject=subject_data,
    body=body_data,
    to_email="oktawiusz.receiver@gmail.com",
    from_email="oktawiusz.sender@gmail.com"
)


IndexError: list index out of range

In [40]:
dataframe.to_csv("./../data/data_scrapped/reads/"+new_data+"_cleaned.csv", index=False)
# print(dataframe["Parking_Space"].unique())
print(dataframe.sort_values(by="Price", ascending=False).head(10))

                                                   Link    Price  \
3727  https://www.olx.plhttps://www.otodom.pl/pl/ofe...  999 000   
3740  https://www.olx.plhttps://www.otodom.pl/pl/ofe...  995 000   
1910  https://www.olx.pl/d/oferta/6-pokoi-i-pietro-p...  993 000   
3642  https://www.olx.plhttps://www.otodom.pl/pl/ofe...  990 000   
2222  https://www.olx.plhttps://www.otodom.pl/pl/ofe...  989 000   
3682  https://www.olx.pl/d/oferta/nowoczesne-mieszka...  980 000   
3883  https://www.olx.plhttps://www.otodom.pl/pl/ofe...  980 000   
3676  https://www.olx.plhttps://www.otodom.pl/pl/ofe...  978 500   
3608  https://www.olx.plhttps://www.otodom.pl/pl/ofe...  975 000   
3665  https://www.olx.plhttps://www.otodom.pl/pl/ofe...  950 000   

                  Location   Area      City       Scrapping_Date  \
3727     Warszawa, Ursynów  63.00  warszawa  2025-07-17 01:58:13   
3740       Warszawa, Praga  64.58  warszawa  2025-07-17 01:58:13   
1910  Wrocław, Śródmieście  82.78   wroclaw  20